In [ ]:
!pip install feedparser

In [ ]:
import boto3
import feedparser
import re
import requests
from tqdm.notebook import tqdm

In [ ]:
session = boto3.session.Session(profile_name='dev2.AdministratorAccess')
account_id = session.client('sts').get_caller_identity()['Account']
region = session.region_name

In [ ]:
anchor_rss_url = 'https://anchor.fm/s/6a3312a0/podcast/rss'

In [ ]:
feed = feedparser.parse(anchor_rss_url)

In [ ]:
def sorter(x):
#     print(x, x['episode_number'])
    return 

In [ ]:
episodes = []
for entry in feed['entries']:
    audio_links = [l for l in entry['links'] if l['type'] == 'audio/x-m4a']
    if len(audio_links) > 0:
        title = entry['title']
        episode_number_s = re.sub(r'(\d*)[^\d].*$', r'\1', entry['title'])
        if len(episode_number_s) == 0:
            print(f'WARNING: No episode number found in {title}')
        else:
            episodes.append({
                'audio_url': audio_links[0]['href'],
                'title': entry['title'],
                'episode_number': int(episode_number_s)
            })
episodes.sort(key=lambda x: x['episode_number'])

In [ ]:
episodes

In [ ]:
bucket_name = f'pod-transcription-{account_id}-{region}'

In [ ]:
s3_client = session.client('s3')

In [ ]:
for episode in tqdm(episodes):
    audio_key = f'audio/{episode["episode_number"]}.m4a'
    print(f'Uploading {episode["audio_url"]} to s3://{bucket_name}/{audio_key}')
    resp = requests.get(episode["audio_url"])
    s3_client.put_object(Bucket=bucket_name, Key=audio_key, Body=resp.content)

In [ ]:
dict(item).keys()

In [ ]:
type(item.enclosure)

In [ ]:
item